In [58]:
import os
os.environ['AOC_SESSION'] = '53616c7465645f5fe53b184f367847aa55a2617e1b05248cac098b2c3dd14dae217169b3ce2e42b10bcc2af6d9b6787a4792a557f9705bb1eba86d761f2407d2'

In [2]:
import aocd
from aocd.models import Puzzle
day = 19
year = 2022
puzzle = Puzzle(year=year, day=day)
# data = aocd.get_data(day=day, year=year)
with open('./data/input_{:02d}'.format(day), 'w') as fh:
    fh.write(puzzle.input_data)

In [396]:
from heapq import heappop, heappush
import re
from collections import defaultdict
import bisect
from functools import cache

In [4]:
test_data = """Blueprint 1: Each ore robot costs 4 ore. Each clay robot costs 2 ore. Each obsidian robot costs 3 ore and 14 clay. Each geode robot costs 2 ore and 7 obsidian.
Blueprint 2: Each ore robot costs 2 ore. Each clay robot costs 3 ore. Each obsidian robot costs 3 ore and 8 clay. Each geode robot costs 3 ore and 12 obsidian."""

In [59]:
data = puzzle.input_data.splitlines()
data_test = test_data.splitlines()
len(data), data[:10]

(30,
 ['Blueprint 1: Each ore robot costs 4 ore. Each clay robot costs 3 ore. Each obsidian robot costs 3 ore and 18 clay. Each geode robot costs 4 ore and 8 obsidian.',
  'Blueprint 2: Each ore robot costs 4 ore. Each clay robot costs 4 ore. Each obsidian robot costs 2 ore and 11 clay. Each geode robot costs 2 ore and 7 obsidian.',
  'Blueprint 3: Each ore robot costs 4 ore. Each clay robot costs 4 ore. Each obsidian robot costs 4 ore and 5 clay. Each geode robot costs 2 ore and 10 obsidian.',
  'Blueprint 4: Each ore robot costs 3 ore. Each clay robot costs 3 ore. Each obsidian robot costs 3 ore and 8 clay. Each geode robot costs 2 ore and 12 obsidian.',
  'Blueprint 5: Each ore robot costs 4 ore. Each clay robot costs 4 ore. Each obsidian robot costs 4 ore and 17 clay. Each geode robot costs 4 ore and 16 obsidian.',
  'Blueprint 6: Each ore robot costs 2 ore. Each clay robot costs 4 ore. Each obsidian robot costs 4 ore and 20 clay. Each geode robot costs 4 ore and 18 obsidian.',
  '

In [6]:
def parse(lines):
    result = []
    for line in lines:
        ans = list(map(int, re.findall('\d+', line)))
        result.append({
            'index': ans[0],
            'robots': [(ans[1], 0, 0, 0), (ans[2], 0, 0, 0), (ans[3], ans[4], 0, 0), (ans[5], 0, ans[6], 0)]
        })
    return result

In [7]:
def try_blueprint(bp, max_minutes=24):
    # prio, (minutes, geodes, ore, clay, obsidian, ore-robots, clay-robots, obsidian-robots, geode-robots)
    cost = bp['robots']
    max_cost = [max(i) for i in zip(*cost)]
    print(cost, max_cost)
    
    start = (0, (0, (0, 0, 0, 0), (1, 0, 0, 0)), [])

In [ ]:
[(4, 0, 0, 0), (2, 0, 0, 0), (3, 14, 0, 0), (2, 0, 7, 0)] [4, 14, 7, 0]

In [970]:
def try_blueprint(bp, only_increasing=False, target=None, max_minutes=24, seen=None, always_build_best=False):
    # prio, (minutes, geodes, ore, clay, obsidian, ore-robots, clay-robots, obsidian-robots, geode-robots)
    # start  = ((0,0,0,0,0,-1), (0, (0, 0, 0, 0), (1, 0, 0, 0), 1), [])
    # start2 = ((0,0,0,0,0,-1), (0, (0, 0, 0, 0), (1, 0, 0, 0), 0), [])
    start  = (0, (0, (0, 0, 0, 0), (1, 0, 0, 0), 1), [])
    start2 = (0, (0, (0, 0, 0, 0), (1, 0, 0, 0), 0), [])
    cost = bp['robots']
    max_cost = [max(i) for i in zip(*cost)]
    print(cost, max_cost)
    ore_cost = cost[0][0]
    clay_cost = cost[1][0] * ore_cost
    obs_cost = cost[2][0] * ore_cost + cost[2][1]*clay_cost
    geo_cost = cost[3][0] * ore_cost + cost[3][2]*obs_cost
    
    geodes_cumsum = [sum(range(i+1)) for i in range(0, 33)]
    
    def score(res):
        sc = res[0] + res[1] * cost[1][0] + res[2] * cost[2][1] * cost[1][0] + res[3] * cost[3][3] * cost[2][1] * cost[1][0]
        return sc
        
    pruned = 0 
    
    best = 0
    best_all = None
    
    def push_stack(stack, minutes, resources, robots, planned, other):
        # Other than that I came up with a scoring function where I take 
        # the current state (items + machines * cycles remaining) and 
        # score it by converting all of them into 'equiverlant ore cost'.
        
        # prio = (-(resources[3] + (max_minutes-minutes) * robots[3]), tuple([-rob for rob in robots[::-1]]),  tuple([-res for res in resources[::-1]]))
        # prio = -(resources[3] + (max_minutes-minutes) * robots[3])
        # prio = -((max_minutes-minutes) * robots[3])
        # prio = -resources[3] + minutes
        
        # prio = tuple([-minutes, -planned, -robots[3], -robots[2], -robots[1], -robots[0]])
        # prio = -minutes
        # prio = (resources[0] // 5) - ((max_minutes-minutes) * robots[3]) - resources[3]
        
        ext_res = [resources[j] + robots[j] * (max_minutes-minutes) for j in range(4)]
        prio = -ext_res[0]*ore_cost - ext_res[1]*clay_cost - ext_res[2]*obs_cost - ext_res[3]*geo_cost
        
        state = (minutes, tuple(resources), tuple(robots), planned)
        # if (minutes == 7): print(state)
        if minutes <= max_minutes:
            heappush(stack, (prio, state, other + [(minutes, resources, robots, planned)]))
            # stack.append((prio, state, other + [(minutes, resources, robots)]))
            return True
        return False
    
    results = []
    
    if seen is None:
        seen = defaultdict(list)
    remove = set()

    stack = [start, start2]
    while stack:
        prio, state, other = heappop(stack)
        # prio, state, other = stack.pop()
        
        minutes, resources, robots, planned = state
        min2build = max_minutes - minutes - 1
        
        lookup = (minutes, planned)
        
        cmp_res = resources[::-1]
        cmp_rob = robots[::-1]

        if lookup in seen:
            start_index = bisect.bisect(seen[lookup], (cmp_res, cmp_rob))
            # start_index = bisect.bisect(seen[lookup], cmp_rob)
            for res, rob in seen[lookup][start_index:]:
            # for rob in seen[lookup][start_index:]:
                if all([res[j] >= cmp_res[j] for j in range(4)]) and all([rob[j] >= cmp_rob[j] for j in range(4)]):
                # if all([rob[j] >= cmp_rob[j] for j in range(4)]) and (sum(rob) > sum(cmp_rob)):
                    # print(f'pruned: {minutes} | {cmp_res} <  {res} & {cmp_rob} < {rob}')
                    pruned += 1
                    prune = True
                    break
            #     elif all([res[j] <= cmp_res[j] for j in range(4)]) and all([rob[j] <= cmp_rob[j] for j in range(4)]):
            #         remove.add((res, rob))
            # if remove:
            #     seen[lookup] -= remove
            
        # if prune:
        #     continue
            
        # if minutes < max_minutes:
        #     bisect.insort(seen[lookup], (cmp_res, cmp_rob))
            # bisect.insort(seen[lookup], cmp_rob)
            
        if len(seen[lookup]) > 100000:
            print('!!!')
            return results, seen, pruned
        
        resources = list(resources)
        robots = list(robots)
        minutes += 1
        time_left = max_minutes - minutes
        
        max_geodes = resources[3] + (robots[3] * (time_left+1)) + geodes_cumsum[time_left]
        
        if max_geodes < best:
            continue
        
        if minutes > max_minutes:
            
            if resources[3] > best:
                best = resources[3]
                best_all = (state, other)
                
            results.append((state, other))
            if len(results) > 300000:
                print(pruned)
                return results, seen
            continue
            return results
        
        new_robots = [0, 0, 0, 0]
        
        if all([cost[planned][j] <= resources[j] for j in range(4)]):
            resources = [resources[j] - cost[planned][j] for j in range(4)]
            new_robots[planned] += 1

            for i in range(planned if only_increasing else 0, 4):
                
                if (time_left <= 1) and (i<3):
                    continue
                
                if (time_left <= 2) and (i<2):
                    continue 
                    
                if (time_left <= 3) and (i<1):
                    continue
                
                if (robots[i] >= max_cost[i]) and (i<3):
                    continue
                
                # if ((max_cost[i] * min2build) <= (resources[i] + (robots[i] * min2build))) and (i<3):
                #     continue
                    
                # if all([resources[j] - robots[j] for j in range(4)])
                
                if target is not None and ((robots[i] + new_robots[i]) > target[i]) and (i<3):
                    # print('skipping', i, robots, target)
                    continue
                    
                if always_build_best:
                    new_resources = tuple([resources[j] + robots[j] for j in range(4)])
                    if all([cost[3][j] <= new_resources[j] for j in range(4)]) and (i<3):
                        continue
                    # if all([cost[2][j] <= new_resources[j] for j in range(4)]) and (i<2):
                    #     continue
                    
                if (i in (0, 1)) or ((i==2) and (robots[1]>0)) or ((i==3) and (robots[2]>0)):
                    planned = i
                    new_resources = [resources[j] + robots[j] for j in range(4)]
                    next_robots = [robots[j] + new_robots[j] for j in range(4)]
                    
                    # while not all([cost[planned][j] <= new_resources[j] for j in range(4)]) and (minutes<max_minutes):
                    #     minutes += 1
                    #     new_resources = [new_resources[j] + next_robots[j] for j in range(4)]
                        
                    push_stack(
                        stack,
                        minutes, 
                        new_resources, 
                        next_robots,
                        planned,
                        other
                    )
            
                    
        if not any(new_robots):
            # do nothing
            new_resources = [resources[j] + robots[j] for j in range(4)] 
            
            # while (not all([cost[planned][j] <= new_resources[j] for j in range(4)])) and (minutes<max_minutes):
            #     minutes += 1
            #     new_resources = [new_resources[j] + robots[j] for j in range(4)]
                
            push_stack(
                stack, 
                minutes, 
                new_resources, 
                robots, 
                planned, 
                other
            )
            
        if len(stack) > 1e5:
            print('abort')
            return results, stack, seen, pruned
    print('pruned: ', pruned)    
    return results, seen
    

In [971]:
# blueprints = parse(data)
blueprints = parse(data_test)

In [984]:
%%time
res, seen = try_blueprint(blueprints[1], always_build_best=False)

[(2, 0, 0, 0), (3, 0, 0, 0), (3, 8, 0, 0), (3, 0, 12, 0)] [3, 8, 12, 0]
pruned:  0
CPU times: user 996 ms, sys: 115 µs, total: 997 ms
Wall time: 995 ms


In [985]:
len(res)

688

In [986]:
# res = res[0]
max([r[0][1][3] for r in res])

12

In [405]:
24-17-1

6

In [377]:
seen[(0, 0)]

[(0, 0, 0, 1)]

In [481]:
for r in res:
    if r[0][1][3] == 9:
        x = r
        # print(x)
x

((24, (4, 46, 8, 9), (1, 5, 2, 2), (24, 74, 22, 9), 0),
 [(1, [1, 0, 0, 0], [1, 0, 0, 0], 1),
  (2, [2, 0, 0, 0], [1, 0, 0, 0], 1),
  (3, [1, 0, 0, 0], [1, 1, 0, 0], 1),
  (4, [2, 1, 0, 0], [1, 1, 0, 0], 1),
  (5, [1, 2, 0, 0], [1, 2, 0, 0], 1),
  (6, [2, 4, 0, 0], [1, 2, 0, 0], 1),
  (7, [1, 6, 0, 0], [1, 3, 0, 0], 2),
  (8, [2, 9, 0, 0], [1, 3, 0, 0], 2),
  (9, [3, 12, 0, 0], [1, 3, 0, 0], 2),
  (10, [4, 15, 0, 0], [1, 3, 0, 0], 2),
  (11, [2, 4, 0, 0], [1, 3, 1, 0], 1),
  (12, [1, 7, 1, 0], [1, 4, 1, 0], 2),
  (13, [2, 11, 2, 0], [1, 4, 1, 0], 2),
  (14, [3, 15, 3, 0], [1, 4, 1, 0], 2),
  (15, [1, 5, 4, 0], [1, 4, 2, 0], 3),
  (16, [2, 9, 6, 0], [1, 4, 2, 0], 3),
  (17, [3, 13, 8, 0], [1, 4, 2, 0], 3),
  (18, [2, 17, 3, 0], [1, 4, 2, 1], 1),
  (19, [1, 21, 5, 1], [1, 5, 2, 1], 3),
  (20, [2, 26, 7, 2], [1, 5, 2, 1], 3),
  (21, [1, 31, 2, 3], [1, 5, 2, 2], 0),
  (22, [2, 36, 4, 5], [1, 5, 2, 2], 0),
  (23, [3, 41, 6, 7], [1, 5, 2, 2], 0),
  (24, [4, 46, 8, 9], [1, 5, 2, 2], 0)])

In [156]:
results, stack, seen, pruned = res
pruned

0

In [170]:
resources = [2, 9, 3, 0]
robots = [2, 3, 1, 0]
cmp_res = resources[::-1]
cmp_rob = robots[::-1]
lookup = (14, 3)

if lookup in seen:
    start_index = bisect.bisect(seen[lookup], (cmp_res, cmp_rob))
    for res, rob in seen[lookup][start_index:]:
        if all([res[j] >= cmp_res[j] for j in range(4)]) and all([rob[j] >= cmp_rob[j] for j in range(4)]):
            print(res, rob)

In [175]:
seen[(13,1)]

[((0, 0, 8, 3), (0, 1, 3, 3)),
 ((0, 0, 9, 3), (0, 1, 3, 2)),
 ((0, 0, 10, 1), (0, 1, 3, 1)),
 ((0, 0, 10, 3), (0, 1, 4, 3)),
 ((0, 0, 11, 3), (0, 1, 4, 2)),
 ((0, 0, 11, 3), (0, 1, 5, 3)),
 ((0, 0, 15, 3), (0, 1, 5, 3)),
 ((0, 0, 16, 3), (0, 0, 4, 3)),
 ((0, 0, 16, 3), (0, 1, 5, 2)),
 ((0, 0, 17, 3), (0, 0, 3, 4)),
 ((0, 0, 17, 3), (0, 0, 4, 2)),
 ((0, 0, 18, 1), (0, 0, 4, 1)),
 ((0, 0, 18, 3), (0, 0, 3, 2)),
 ((0, 1, 6, 3), (0, 2, 4, 3)),
 ((0, 1, 7, 3), (0, 1, 3, 3)),
 ((0, 1, 7, 3), (0, 2, 5, 3)),
 ((0, 1, 9, 3), (0, 1, 4, 3)),
 ((0, 1, 10, 3), (0, 1, 5, 3)),
 ((0, 1, 14, 3), (0, 1, 5, 3)),
 ((0, 2, 4, 3), (0, 2, 4, 2)),
 ((0, 2, 6, 3), (0, 2, 4, 2)),
 ((0, 2, 6, 3), (0, 2, 5, 3)),
 ((0, 2, 7, 1), (0, 1, 3, 1)),
 ((0, 2, 7, 3), (0, 1, 2, 2)),
 ((0, 2, 7, 3), (0, 1, 2, 4)),
 ((0, 2, 7, 3), (0, 1, 3, 2)),
 ((0, 2, 7, 3), (0, 1, 4, 2)),
 ((0, 2, 9, 3), (0, 1, 3, 4)),
 ((0, 2, 9, 3), (0, 1, 4, 2)),
 ((0, 2, 12, 3), (0, 1, 5, 2)),
 ((0, 2, 14, 3), (0, 1, 4, 4)),
 ((0, 2, 14, 3), (0, 1, 

In [987]:
%%time
blueprints = parse(data)
ans = 0
for bp in blueprints:
    print('trying ', bp['index'])
    res, _ = try_blueprint(bp, always_build_best=False)    
    print('# ', len(res))
    ngeodes = max([r[0][1][3] for r in res])
    print(f"Max geodes: {ngeodes}")
    ans += bp['index'] * ngeodes
print(ans)

trying  1
[(4, 0, 0, 0), (3, 0, 0, 0), (3, 18, 0, 0), (4, 0, 8, 0)] [4, 18, 8, 0]
pruned:  0
#  176
Max geodes: 2
trying  2
[(4, 0, 0, 0), (4, 0, 0, 0), (2, 11, 0, 0), (2, 0, 7, 0)] [4, 11, 7, 0]
pruned:  0
#  771
Max geodes: 3
trying  3
[(4, 0, 0, 0), (4, 0, 0, 0), (4, 5, 0, 0), (2, 0, 10, 0)] [4, 5, 10, 0]
pruned:  0
#  1142
Max geodes: 5
trying  4
[(3, 0, 0, 0), (3, 0, 0, 0), (3, 8, 0, 0), (2, 0, 12, 0)] [3, 8, 12, 0]
pruned:  0
#  1309
Max geodes: 7
trying  5
[(4, 0, 0, 0), (4, 0, 0, 0), (4, 17, 0, 0), (4, 0, 16, 0)] [4, 17, 16, 0]
pruned:  0
#  4605
Max geodes: 0
trying  6
[(2, 0, 0, 0), (4, 0, 0, 0), (4, 20, 0, 0), (4, 0, 18, 0)] [4, 20, 18, 0]
pruned:  0
#  27853
Max geodes: 0
trying  7
[(3, 0, 0, 0), (4, 0, 0, 0), (2, 20, 0, 0), (4, 0, 7, 0)] [4, 20, 7, 0]
pruned:  0
#  2865
Max geodes: 2
trying  8
[(3, 0, 0, 0), (4, 0, 0, 0), (3, 6, 0, 0), (2, 0, 10, 0)] [4, 6, 10, 0]
pruned:  0
#  1320
Max geodes: 9
trying  9
[(4, 0, 0, 0), (3, 0, 0, 0), (3, 14, 0, 0), (4, 0, 17, 0)] [4, 14, 

In [988]:
# 1588
ans

1613

In [489]:
puzzle.answer_a = ans

That's the right answer!  You are one gold star closer to collecting enough star fruit. [Continue to Part Two]


# Part 2

In [989]:
%%time
blueprints = parse(data_test)
res, seen = try_blueprint(blueprints[0], max_minutes=32, always_build_best=False)

[(4, 0, 0, 0), (2, 0, 0, 0), (3, 14, 0, 0), (2, 0, 7, 0)] [4, 14, 7, 0]
pruned:  0
CPU times: user 15.4 s, sys: 31.3 ms, total: 15.5 s
Wall time: 15.5 s


In [990]:
max([r[0][1][3] for r in res])

56

In [751]:
len(res)

30001

In [991]:
%%time
blueprints = parse(data)
ans = 1
for bp in blueprints[:3]:
    print('trying ', bp['index'])
    res, _ = try_blueprint(bp, max_minutes=32, always_build_best=True)    
    print('# ', len(res))
    ngeodes = max([r[0][1][3] for r in res])
    print(f"Max geodes: {ngeodes}")
    ans *= ngeodes
print(ans)

trying  1
[(4, 0, 0, 0), (3, 0, 0, 0), (3, 18, 0, 0), (4, 0, 8, 0)] [4, 18, 8, 0]
pruned:  0
#  1920
Max geodes: 28
trying  2
[(4, 0, 0, 0), (4, 0, 0, 0), (2, 11, 0, 0), (2, 0, 7, 0)] [4, 11, 7, 0]
pruned:  0
#  747
Max geodes: 38
trying  3
[(4, 0, 0, 0), (4, 0, 0, 0), (4, 5, 0, 0), (2, 0, 10, 0)] [4, 5, 10, 0]
pruned:  0
#  318
Max geodes: 44
46816
CPU times: user 16.3 s, sys: 28.2 ms, total: 16.3 s
Wall time: 16.3 s


In [992]:
print(ans)

46816


In [993]:
puzzle.answer_b = ans

That's the right answer!  You are one gold star closer to collecting enough star fruit.You have completed Day 19! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


In [994]:
%%time
# solution from here: https://www.reddit.com/r/adventofcode/comments/zpihwi/comment/j0tvzgz/?utm_source=share&utm_medium=web2x&context=3

import re, numpy

V = lambda *a: numpy.array(a)
key = lambda a: tuple(a[0]+a[1]) + tuple(a[1])
prune = lambda x: sorted({key(x):x for x in x}.values(), key=key)[-1000:]

def parse(line):
    i,a,b,c,d,e,f = map(int, re.findall(r'\d+',line))
    return (i, (V(0,0,0,a), V(0,0,0,1)),    # Cost and production
               (V(0,0,0,b), V(0,0,1,0)),    # of each robot type,
               (V(0,0,d,c), V(0,1,0,0)),    # in the order geode,
               (V(0,f,0,e), V(1,0,0,0)),    # obs, clay, and ore.
               (V(0,0,0,0), V(0,0,0,0)))    # Construct no robot.

def run(blueprint, t):
    todo = [(V(0,0,0,0), V(0,0,0,1))]       # What we have and make.
    for t_ in range(t,0,-1):
        todo_ = list()                      # Queue for the next minute.
        for have, make in todo:
            for cost, more in blueprint:
                if all(cost <= have):       # We can afford this robot.
                    todo_.append((have+make-cost, make+more))
        todo = prune(todo_)                 # Prune the search queue.
    return max(have[0] for have, _ in todo)

part1, part2 = 0, 1
for i, *blueprint in map(parse, open('./data/input_19')):
    part1 += run(blueprint, 24) * i
    part2 *= run(blueprint, 32) if i<4 else 1

print(part1, part2)

1613 42328
CPU times: user 15.2 s, sys: 44.5 ms, total: 15.3 s
Wall time: 15.1 s
